In [2]:
import os
import cv2
import csv
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors

def get_image_colors(image_path, num_colors=2):
    print(f"Traitement de l'image : {image_path}")

    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)

    # Convertir l'image de BGR à RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Redimensionner l'image pour accélérer le calcul
    resized_image = cv2.resize(image_rgb, (100, 100), interpolation=cv2.INTER_AREA)

    # Aplatir l'image en un tableau 2D
    flattened_image = resized_image.reshape((-1, 3))

    # Utiliser l'algorithme de clustering Mini-Batch K-means pour trouver les couleurs dominantes
    kmeans = MiniBatchKMeans(n_clusters=num_colors, random_state=42, n_init=3)
    kmeans.fit(flattened_image)

    # Obtenir les couleurs dominantes du modèle Mini-Batch K-means
    couleurs_dominantes = kmeans.cluster_centers_
    couleurs_dominantes = couleurs_dominantes.astype(int)

    return couleurs_dominantes

def convertir_en_couleur_nominale(couleur_rgb):
    # Définir une liste de couleurs nominatives
    couleurs_nominatives = ['noir', 'blanc', 'gris', 'rouge', 'vert', 'bleu', 'jaune', 'orange', 'rose', 'violet', 'marron']

    # Définir les valeurs RVB correspondantes aux couleurs nominatives
    couleurs_rgb = np.array([
        [0, 0, 0],     # noir
        [255, 255, 255],   # blanc
        [128, 128, 128],   # gris
        [255, 0, 0],   # rouge
        [0, 255, 0],   # vert
        [0, 0, 255],   # bleu
        [255, 255, 0],   # jaune
        [255, 165, 0],   # orange
        [255, 192, 203],   # rose
        [128, 0, 128],   # violet
        [165, 42, 42]   # marron
    ])

    # Créer un modèle Nearest Neighbors et ajuster les données RVB
    knn = NearestNeighbors(n_neighbors=1)
    knn.fit(couleurs_rgb)

    # Trouver le voisin le plus proche des valeurs RVB données
    _, indices_plus_proches = knn.kneighbors(couleur_rgb)
    indices_plus_proches = indices_plus_proches.flatten()

    # Récupérer les noms de couleur correspondant aux indices
    couleurs_nominatives = [couleurs_nominatives[indice] for indice in indices_plus_proches]

    return couleurs_nominatives

def process_images_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    data = []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image_id = os.path.splitext(image_file)[0]

        try:
            couleurs_dominantes = get_image_colors(image_path)
            
            couleurs_secondaires = []
            for couleur_rgb in couleurs_dominantes:
                couleur_nominale = convertir_en_couleur_nominale([couleur_rgb])
                couleurs_secondaires.extend(couleur_nominale)
            
            couleur_secondaire = ', '.join(couleurs_secondaires[1:])
            data.append({'id': image_id, 'couleur_secondaire': couleur_secondaire})

            
            print(f"Image traitée avec succès : {image_file}")
        except:
            print(f"Erreur lors du traitement de l'image : {image_file}")
    
    #csv_path = os.path.join(folder_path, 'color2.csv')
    with open('color2.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'couleur_secondaire']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for item in data:
            writer.writerow(item)
    
    print("Terminé ! Le fichier CSV a été créé avec succès.")



In [3]:
# Exemple d'utilisation
folder_path = '../../Multi_Label_dataset/Images'
#folder_test = r'C:\Users\Sitti Fatouma\Documents\MSc2\Big Data\Nostradamovies\code propre\img test'
process_images_folder(folder_path)

Traitement de l'image : ../../Multi_Label_dataset/Images\tt0084058.jpg
Image traitée avec succès : tt0084058.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0084867.jpg
Image traitée avec succès : tt0084867.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085121.jpg
Image traitée avec succès : tt0085121.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085154.jpg
Image traitée avec succès : tt0085154.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085159.jpg
Image traitée avec succès : tt0085159.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085208.jpg
Image traitée avec succès : tt0085208.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085236.jpg
Image traitée avec succès : tt0085236.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085244.jpg
Image traitée avec succès : tt0085244.jpg
Traitement de l'image : ../../Multi_Label_dataset/Images\tt0085248.jpg
Image traitée avec succès

In [4]:
color_mapping = {
    'noir': '0',
    'blanc': '1',
    'gris': '2',
    'rose': '3',
    'orange': '4',
    'marron': '5',
    'bleu': '6',
    'violet': '7',
    'rouge': '8',
    'jaune': '9',
    'vert': '10'
}

try:
    with open('color2.csv', 'r') as input_file, open('color2_num.csv', 'w', newline='') as output_file:
        reader = csv.DictReader(input_file)
        fieldnames = reader.fieldnames

        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            couleur_secondaire = row['couleur_secondaire']
            for color, value in color_mapping.items():
                couleur_secondaire = couleur_secondaire.replace(color, value)
            row['couleur_secondaire'] = couleur_secondaire
            writer.writerow(row)

    print("Le fichier color2_num.csv a été créé avec succès.")
except Exception as e:
    print(f"Erreur lors de la création du fichier color2_num.csv : {str(e)}")


Le fichier color2_num.csv a été créé avec succès.
